### imports

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import pandas as pd
import re

import nltk
from nltk import bigrams

### import the data and store it as an array in the variable: corpus

In [2]:
data = pd.read_csv('blogtext_5k.csv')
corpus = data['text'].values

In [3]:
def preprocess(corpus):
    # loop through each document in the corpus
    for i in range(len(corpus)):
        # remove extra whitespace and then lowercase all letters
        temp = re.sub('[^a-z ]','',corpus[i].strip().lower())

        # split and rejoin document to remove extra space within text
        temp = [word for word in temp.split()]
        temp = " ".join(temp)
        corpus[i] = temp
    
    return corpus

In [4]:
corpus = preprocess(corpus)

In [15]:
bi_grams = {}

# loop through each document
for i in range(len(corpus)):
    
    # tokenize the document
    tokens = ['<start>'] + nltk.word_tokenize(corpus[i]) 
    # add start & end tags
    tokens = ['<start>'] + tokens + ['<end>']
    
    # create bi tokens
    bi_tokens = bigrams(tokens)    

    # add the bi tokens to the full list of bi grams
    for bi_token in bi_tokens:
        if bi_token not in bi_grams:
            bi_grams[bi_token] = 1
        else:
            bi_grams[bi_token] += 1
            


### create a dataframe to store the results
<b>pre</b> contains the first word, <b>post</b> contains the second work, and <b>count</b> contains the count of the occurences of the two words

In [19]:
# create a dataframe to store the results
reults = pd.Series(bi_grams).reset_index().rename(
    {'level_0':'pre','level_1':'post',0:'count'},axis=1
).set_index('pre')

reults.head()

,post,count
pre,,
<start>,<start>,5000
<start>,info,1
info,has,1
has,been,235
been,found,2


### here, we generate a sentence of up to 10 words

In [27]:
# initialize the sentence with the <start> tag
current_word = '<start>'
sentence = current_word

for i in range(10):
    
    # when there's no next word prediction, end the loop
    if (reults.index==current_word).sum() == 0 or current_word == '<end>':
        break
        
    # when there's only one possible word, set it as the prediction
    if (reults.index==initial).sum() == 1:
        initial = reults.loc[initial]['post']
        
    # when there's more than one possible word, take the top 10 words and do a fitness proportionate selection
    # Fitness proportionate selection: https://en.wikipedia.org/wiki/Fitness_proportionate_selection
    else:
        temp = reults.loc[initial].sort_values('count',ascending=False).iloc[:10]
        temp['count'] = (temp['count'] / temp['count'].sum()).cumsum()
        rnd = np.random.sample()

        temp['bullseye'] = temp['count']>rnd

        initial = temp[temp['bullseye']]['post'].iloc[0]
    
    sentence += ' '+initial
    
print(sentence.replace('<start> ','').replace('<end> ',''))

in the last night and the first place in the


### results: since we used bi grams, we find that when considering each 2 words separately, they make sense grammatically. However, the full sentence is usually incorrect 